In [1]:

import torch
import scanpy as sc
sn_data = sc.read_h5ad('/cluster/home/sunyk/deeplearning/.sun_algo/test_data/reference.h5ad')
st_data = sc.read_h5ad('/cluster/home/sunyk/deeplearning/.sun_algo/test_data/query.h5ad')

sn_data.var_names = sn_data.var['features']
st_data.var_names = st_data.var['features']

In [2]:
import os
os.chdir('/cluster/home/sunyk/deeplearning/.sun_algo/rmge')
from preprocess import preprocess_data, generate_noisy_data

st_data, sn_data = preprocess_data(st_data, sn_data)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
noise_data = generate_noisy_data(st_data, sn_data, device)
sn_data.layers['noise']=noise_data.cpu().numpy()
# sn_data.write('/cluster/home/sunyk/deeplearning/GLUE-master1/GLUE-master/output/noise_sub_cell.h5ad')

/cluster/home/sunyk/anaconda3/envs/cuda6/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [3]:

from model import RMGE 
        
rmge = RMGE(sn_data, st_data, device='cuda',class_columns = 'SubClass')

# 训练模型
train_losses, train_accuracies = rmge.train(epochs=1000)

# 训练过程结束后评估模型
st_predict_class_names = rmge.evaluate()

Epoch 0/1000, Loss: 18.5913028717041, Accuracy: 0.1488370965003759
Epoch 100/1000, Loss: 2.0847513675689697, Accuracy: 0.9104115635534644
Epoch 200/1000, Loss: 2.0579540729522705, Accuracy: 0.9106783401644314
Epoch 300/1000, Loss: 2.069488525390625, Accuracy: 0.910993621613756
Epoch 400/1000, Loss: 2.077226400375366, Accuracy: 0.9099265151698882
Epoch 500/1000, Loss: 2.1036317348480225, Accuracy: 0.91111488370965
Epoch 600/1000, Loss: 2.114243984222412, Accuracy: 0.910217544200034
Epoch 700/1000, Loss: 2.1341300010681152, Accuracy: 0.9115029224165111
Epoch 800/1000, Loss: 2.1573081016540527, Accuracy: 0.9105570780685374
Epoch 900/1000, Loss: 2.1814985275268555, Accuracy: 0.9090776804986297
Validation Accuracy: 0.8948


In [4]:
from mapping import cell_mappings 
    
cell_mappings, total_transport_cost = cell_mappings(sn_data, st_data, rmge, device)

# Print the total transport cost
print("Total Transport Cost:", total_transport_cost)


Total Transport Cost: 402.60230281782685
